Retrieval-Augmented Generation (RAG) combines a retrieval system, which fetches relevant documents, with a generative model, allowing it to incorporate external knowledge for more accurate and informed responses. This notebook shows how to use the CrateDB vector store functionality to create a retrieval augmented generation (RAG) pipeline.

## What is CrateDB?

CrateDB is an open-source, distributed, and scalable SQL analytics database for storing and analyzing massive amounts of data in near real-time, even with complex queries. It is wire-compatible to PostgreSQL, based on Lucene, and inherits the shared-nothing distribution layer of Elasticsearch.

This example uses the Python client driver for CrateDB and vector store support in LangChain.

## Getting Started
CrateDB supports storing vectors since version 5.5. You can leverage the fully managed service of CrateDB Cloud, or install CrateDB on your own, for example using Docker.

```shell
docker run --publish 4200:4200 --publish 5432:5432 --pull=always crate:latest -Cdiscovery.type=single-node
```

## Setup

Install required Python packages, and import Python modules.

In [1]:
!pip install -r requirements.txt

  Cloning https://github.com/crate-workbench/langchain.git (to revision cratedb) to /private/var/folders/3f/htk34xrs62d0jxkjddpz35qc0000gn/T/pip-install-xz119_ob/langchain_9590a3e7f7484f648de0d811086d8e7c
  Running command git clone --filter=blob:none --quiet https://github.com/crate-workbench/langchain.git /private/var/folders/3f/htk34xrs62d0jxkjddpz35qc0000gn/T/pip-install-xz119_ob/langchain_9590a3e7f7484f648de0d811086d8e7c
  Resolved https://github.com/crate-workbench/langchain.git to commit 5df2429aa2fec83b424cf21bc190f8bc9c36845b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/crate-workbench/langchain.git (to revision cratedb) to /private/var/folders/3f/htk34xrs62d0jxkjddpz35qc0000gn/T/pip-install-xz119_ob/langchain-community_6ba38b38738a48bfb9f485a33e88c70d
  Running command git clone --filter=blob:none --quiet https://github.com/crate-workbench/langchain.git /priv

In [2]:
import os

import openai
import pandas as pd
import sqlalchemy as sa

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from pueblo.util.environ import getenvpass
from langchain.docstore.document import Document
from langchain.vectorstores import CrateDBVectorSearch

ImportError: cannot import name 'CrateDBVectorSearch' from 'langchain.vectorstores' (/opt/homebrew/lib/python3.10/site-packages/langchain/vectorstores/__init__.py)

### Configure database settings

This notebook will connect to a CrateDB server instance running on localhost. You can start a sandbox instance on your workstation by running [CrateDB using Docker]. Alternatively, you can also connect to a cluster running on [CrateDB Cloud].

[CrateDB Cloud]: https://console.cratedb.cloud/
[CrateDB using Docker]: https://crate.io/docs/crate/tutorials/en/latest/basic/index.html#docker.

In [3]:
# Define the connection string to running CrateDB instance.
CONNECTION_STRING = os.environ.get(
    "CRATEDB_CONNECTION_STRING",
    "crate://crate@localhost/",
)

# Define the store collection to use for this notebook session.
COLLECTION_NAME = "customer_data"

### Configure OpenAI

In this example you need to have an API key from OpenAI. This is typically done by creating an account on OpenAI's website and accessing the API section, where you can generate a new key.

In [4]:
getenvpass("OPENAI_API_KEY", prompt="OpenAI API key:")
openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API key:········


### Patches
Those can be removed again after they have been upstreamed.

In [5]:
from cratedb_toolkit.sqlalchemy.patch import patch_inspector
patch_inspector()

## Create embeddings from dataset

We use `CSVLoader` class to load support tickets from Twitter. The next step initializes a vector search store in CrateDB using embeddings generated by an OpenAI model. This will create a table that stores the embeddings with the name of the collection. Make sure the collection name is unique and that you have the permission to create a table.

In [6]:
loader = CSVLoader(file_path="./sample_data/twitter_support_microsoft.csv", encoding="utf-8", csv_args={'delimiter': ','})
data = loader.load()

In [7]:
embeddings = OpenAIEmbeddings()

store = CrateDBVectorSearch.from_documents(
    embedding=embeddings,
    documents=data,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
)

/opt/homebrew/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


NameError: name 'CrateDBVectorSearch' is not defined

## Ask question
Let's define our question:

In [ ]:
my_question = "How to update shipping address on existing order in Microsoft Store?"

## Find relevant context using similarity search

The similarity search uses Eucledian distance to find similar vectors and compute the score:

In [ ]:
docs_with_score = store.similarity_search_with_score(my_question)
documents=[]
for doc, score in docs_with_score:
    documents.append(doc.page_content)

## Augment system prompt and query LLM

In the final step we create an interactive chatbot scenario where GPT-4 serves as a customer support assistant, using a given set of documents as its knowledge base to answer questions about Microsoft products and services. If the answer to a question isn't in the provided documents, it's programmed to respond with "I don't know."

In [ ]:
context = '---\n'.join(documents)

system_prompt = f"""
You are customer support expert and get questions about Microsoft products and services.
To answer question use the information from the context. Remove new line characters from the answer.
If you don't find the relevant information there, say "I don't know".

Context:
{context}"""

chat_completion = openai.chat.completions.create(model="gpt-3.5-turbo",
                                               messages=[{"role": "system", "content": system_prompt},
                                                         {"role": "user", "content": my_question}])


In [ ]:
chat_completion.choices[0].message.content